In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from pandas import DataFrame
import plotly
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df1= pd.read_csv("E:\\python\\Internship_themlco\\Data in Brief\\WildBlueberryPollinationSimulationData.csv")

In [ ]:
df1.head()

In [ ]:
df1

In [ ]:
df1.info()

In [ ]:
df1.nunique()

In [ ]:
df1.shape

In [ ]:
df1.isnull()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.describe()

In [ ]:
df1.columns

In [ ]:
df1.drop('Row#', axis='columns', inplace=True)

In [ ]:
sns.pairplot(df1)
plt.show()

In [ ]:
sns.displot(df1, x="yield")
plt.figure(figsize=(8, 6))

In [ ]:
sns.boxplot(x=df1['bumbles'])
plt.figure(figsize=(8, 6))

In [ ]:
sns.boxplot(x=df1['honeybee'])
plt.figure(figsize=(8, 6))

In [ ]:
sns.boxplot(x=df1['andrena'])
plt.figure(figsize=(8, 6))

In [ ]:
sns.boxplot(x=df1['osmia'])
plt.figure(figsize=(8, 6))

In [ ]:
profile = ProfileReport(df1, title="Yield detection", explorative=True)
profile

In [ ]:
q1 = df1.quantile(0.25)
q2 = df1.quantile(0.75)
iqr = q2 -q1
print(iqr)

In [ ]:
df1 = df1[~((df1 < (q1 - 1.5 * iqr)) |(df1 > (q2 + 1.5 * iqr))).any(axis=1)]
df1.shape

In [ ]:
corelation = df1.corr()  

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sns.heatmap(df1.corr(), annot = True)
plt.title('Heatmap to understand the correlation between input data', fontsize=10)

In [ ]:
X = df1[['clonesize','honeybee','bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange', 'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']]
X

In [ ]:
y = df1['yield']
y

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split,cross_validate

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)

Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [ ]:
reg.fit(X_train, Y_train)

In [ ]:
Y_pred = reg.predict(X_test)

In [ ]:
rmse = np.sqrt(np.mean((Y_test - Y_pred)**2))        
rmse

In [ ]:
from sklearn.metrics import r2_score
r2_score(Y_pred, Y_test)

In [ ]:
data_pred = X_test.iloc[8,:] 
data_pred_array = data_pred.values.reshape(1, -1)


reg.predict(data_pred_array)

In [ ]:
rg = RandomForestRegressor(random_state = 20)
rg.fit(X_train, Y_train)

y_pred = rg.predict(X_test)
print('''Evaluating Model Performance''')
rg.score(X_test, Y_test)*100

In [ ]:
params = {
    'n_estimators': [50,100,150,200],
    'max_depth': [2,4,6,8,10],
}
model1 = GridSearchCV(RandomForestRegressor(random_state = 1), params)
model1.fit(X_train, Y_train)

y_pred = model1.predict(X_test)
print('''Evaluating Model Performance''')
model1.score(X_test, Y_test)*100

In [ ]:
rmse = np.sqrt(np.mean((Y_test - Y_pred)**2))

print("RMSE : {:.2f}".format(rmse))

In [ ]:
r2_score(Y_pred, Y_test)

In [ ]:
from xgboost import XGBRegressor
regressor = XGBRegressor()
regressor.fit(X_train, Y_train)
Y_pred = regressor.predict(X_test)

In [ ]:
rmse = np.sqrt(np.mean((Y_test - Y_pred)**2))
print("RMSE : {:.2f}".format(rmse))

In [ ]:
r2_score(Y_pred, Y_test)

# Explainable AI

In [ ]:
!pip install shap

In [ ]:
!pip install dabl

In [ ]:
!pip install delayed

In [ ]:
from joblib import Parallel, delayed, parallel_backend

In [ ]:
import numba

In [ ]:
import shap

In [ ]:
from shap import TreeExplainer

In [ ]:
shap_values = shap.TreeExplainer(regressor).shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
exp = shap.Explainer(reg, X_train)
shap_values = exp.shap_values(data_pred)

In [ ]:
shap.initjs()
shap.force_plot(exp.expected_value, shap_values, data_pred)

In [ ]:
# use Kernel SHAP to explain test set predictions
k_exp = shap.KernelExplainer(reg.predict(X_train))
k_shap_values = k_exp.shap_values(data_pred)
shap.force_plot(k_exp.expected_value[1], k_shap_values[1], data_pred)